<a href="https://colab.research.google.com/github/seismic2020/WG1-P4/blob/Edits/WG1P4_SharedAnalyses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Before running this code, check https://github.com/seismic2020/WG1-P4/blob/master/Analysis-Workflow.md for variable naming and data cleaning steps.

In [0]:
# Load libraries
if (!require("pacman")) install.packages("pacman")
library(pacman)
pacman::p_load("tidyverse", "psych", "data.table",  # Data wrangling and descriptive stats
               "epiDisplay", "sjstats")             # Reporting Odds Ratio and Std. Betas

In [0]:
# Load full dataset
df_full <- read.csv("~/YOUR FILE PATH HERE.csv")

In [0]:
# View data 
# Note: Variable names should follow SEISMIC conventions found here: https://docs.google.com/spreadsheets/d/1SzU4PcIEUsAGnKKyAcugHO2O2aZW29sf9a_cC-FAElk/edit#gid=1679989021
names(df_full)
head(df_full)

In [0]:
# Filter for student level inclusion/exclusion criteria
df_full <- df_full %>%
  # Include
  filter(transfer == 0) %>%
  filter(tookcourse_2 == 1) %>%
  filter(cohort >= 2013 & cohort <= 2018) %>%
  # Exclude
  filter(international == 0)

In [0]:
# Create subset dataframes for each analysis sample (for each discipline)

In [0]:
# Bio
# Took 2nd course in sequence
df_bio2 <- df_clean %>%
  subset(discipline == "BIO") %>%
  subset(apyear >= 2013)
# Took AP 
df_bio_aptakers <- df_clean %>%
  subset(discipline == "BIO") %>%
  subset(aptaker == 1)
# Skip eligible
df_bio_skeligible <- df_clean %>%
  subset(discipline == "BIO") %>%
  subset(eligible_to_skip == 1)

# Chem
# Took 2nd course in sequence
df_chem2 <- df_clean %>%
  subset(discipline == "CHEM") %>%
  subset(apyear >= 2013)
# Took AP 
df_chem_aptakers <- df_clean %>%
  subset(discipline == "CHEM") %>%
  subset(aptaker == 1)
# Skip eligible
df_chem_skeligible <- df_clean %>%
  subset(discipline == "CHEM") %>%
  subset(eligible_to_skip == 1)

# Phys
# Took 2nd course in sequence
df_phys2 <- df_clean %>%
  subset(discipline == "PHYS") %>%
  subset(apyear >= 2013)
# Took AP 
df_phys_aptakers <- df_clean %>%
  subset(discipline == "PHYS") %>%
  subset(aptaker == 1)
# Skip eligible
df_phys_skeligible <- df_clean %>%
  subset(discipline == "PHYS") %>%
  subset(eligible_to_skip == 1)


In [0]:
# Run Models (for each discipline)
# Note: For model specifications, check: https://docs.google.com/spreadsheets/d/1rN8W_iz1mr7lEzBGfdTZHa45wKOSLiSF8VEpChCPsmE/edit#gid=129222174

In [0]:
# Bio
# RQ1a - Who takes AP?
bio_rq1a <- glm(aptaker ~ factor(firstgen) + factor(lowincomeflag) + factor(gender) + factor(ethniccode_cat) +
                  scale(hsgpa) + scale(mathsr) + scale(englsr) + factor(cohort), 
                  data=df_bio2, binomial(link="logit"))
summary(bio_rq1a)
robustse(bio_rq1a, coef="odd.ratio") #robustse is a command from sjstats package
logistic.display(bio_rq1a)

In [0]:
# RQ1b - Who gets what score on AP exams? (DV: apscore, Sample: Took AP)
bio_rq1b <- lm(scale(apscore) ~ factor(firstgen) + factor(lowincomeflag) + factor(gender) + factor(ethniccode_cat) +
               scale(hsgpa) + scale(mathsr) + scale(englsr) + factor(cohort), 
               data=df_bio_takers)
summary(bio_rq1b)
std_beta(bio_rq1b) # from sjstats package

In [0]:
# RQ1c.i - Of those who take AP, who earns an AP score that makes them eligible to skip?
bio_rq1ci <- glm(eligible_to_skip ~ factor(firstgen) + factor(lowincomeflag) + factor(gender) + factor(ethniccode_cat) +
               scale(hsgpa) + scale(mathsr) + scale(englsr) + factor(crs_term), 
                data=df_bio_takers, 
                binomial(link="logit"))
summary(bio_rq1ci)
logistic.display(bio_rq1ci)
#robustse(rq1ci, coef="odd.ratio")

In [0]:
# RQ1c.ii - Of everyone, who earns an AP score that makes them eligible to skip? 
bio_rq1cii <- glm(eligible_to_skip ~ factor(firstgen) + factor(lowincomeflag) + factor(gender) + factor(ethniccode_cat) +
               scale(hsgpa) + scale(mathsr) + scale(englsr) + factor(crs_term), 
                data=df_bio2, binomial(link="logit"))
summary(bio_rq1cii)
logistic.display(bio_rq1cii)
#robustse(bio_rq1cii, coef="odd.ratio")

In [0]:
# RQ2a - Of everyone who was eligible to skip, who actually skipped?
bio_rq2a <- glm(eligible_to_skip ~ factor(firstgen) + factor(lowincomeflag) + factor(gender) + factor(ethniccode_cat) +
               hsgpa_z + scale(mathsr) + scale(englsr) + factor(crs_term), 
               data=df_bio_eligible, binomial(link="logit"))
summary(bio_rq2a)
logistic.display(bio_rq2a)
#robustse(bio_rq2a, coef="odd.ratio")

In [0]:
# RQ2b - Of those who were eligible to skip, what was the effect of skipping on 2nd course grade (controlling for AP score)?
bio_rq2b <- lm(scale(numgrade) ~ factor(skipped_course) + scale(apscore) + 
              factor(firstgen) + factor(lowincomeflag) + factor(gender) + factor(ethniccode_cat) + 
              scale(hsgpa) + scale(mathsr) + scale(englsr) + factor(crs_term), 
              data=df_bio_aptaker)
summary(bio_rq2b)
#std_beta(bio_rq2b)

In [0]:
# RQ2c - Of everyone, what grade did they get in the 2nd course (controlling for AP score)? 
bio_rq2c <- lm(scale(numgrade) ~ factor(skipped_course) + scale(apscore) + 
              factor(firstgen) + factor(lowincomeflag) + factor(gender) + factor(ethniccode_cat) + 
              scale(hsgpa) + scale(mathsr) + scale(englsr) + factor(crs_term), 
              data=df_bio2)
summary(bio_rq2c)
#std_beta(bio_rq2c)

In [0]:
# RQ2d - Of everyone who took AP, what grade did they get in the 2nd course?
chemsr <- lm(scale(numgrade) ~ factor(skipped_course) + 
              factor(firstgen) + factor(lowincomeflag) + factor(gender) + factor(ethniccode_cat) + 
              scale(hsgpa) + scale(mathsr) + scale(englsr) + factor(crs_term), 
              data=df_bio_aptakers)
summary(chemsr)
#std_beta(chemsr)

In [0]:
# RQ2e - Of everyone who took AP and skipped, what grade did they get in the 2nd course?
chemsr2 <- lm(numgrade ~ aprealskipper_GChem1 + factor(firstgen) + 
    factor(lowincomeflag) + factor(gender) + factor(ethniccode_cat) + 
    hsgpa_z + scale(mathsr) + scale(englsr) + factor(crs_term), data=chemap_unique)
summary(chemsr2)

#std_beta(chemsr2)

In [0]:
# RQ2f - Course grade, Skip eligible for those who took AP
chemsr3 <- lm(numgrade~apskipper_GChem1 + factor(firstgen) + 
    factor(lowincomeflag) + factor(gender) + factor(ethniccode_cat) + 
    hsgpa_z + scale(mathsr) + scale(englsr) + factor(crs_term), data=chem_tookap)
summary(chemsr3)

#std_beta(chemsr3)

In [0]:
# RQ2g -  Skip eligible, full sample
chemsr4<- lm(numgrade~apskipper_GChem1 + factor(firstgen) + 
    factor(lowincomeflag) + factor(gender) + factor(ethniccode_cat) + 
    hsgpa_z + scale(mathsr) + scale(englsr) + factor(crs_term), data=chemap_unique)
summary(chemsr4)

#std_beta(chemsr4)

In [0]:
## PHYSICS
#table(apfull_SkipEl$apyear_Mec) # NA not coded
apfull_SkipEl$apyear_Mec <- ifelse(is.na(apfull_SkipEl$apyear_Mec), 0, apfull_SkipEl$apyear_Mec)
apfull_SkipEl$apyear_Elec <- ifelse(is.na(apfull_SkipEl$apyear_Elec), 0, apfull_SkipEl$apyear_Elec)

table(apfull_SkipEl$apyear_Mec)
table(apfull_SkipEl$apyear_Elec)

In [0]:
# Chem
# RQ1a - Who takes AP?
bio_rq1a <- glm(aptaker ~ factor(firstgen) + factor(lowincomeflag) + factor(gender) + factor(ethniccode_cat) +
                  scale(hsgpa) + scale(mathsr) + scale(englsr) + factor(cohort), 
                  data=df_bio2, binomial(link="logit"))
summary(bio_rq1a)
robustse(bio_rq1a, coef="odd.ratio") #robustse is a command from sjstats package
logistic.display(bio_rq1a)